In [ ]:
import os
import pickle
from tqdm import tqdm, trange
from dataclasses import dataclass
import pandas as pd
import json
from transformers import AutoTokenizer

In [ ]:
@dataclass
class Dataset:
    dataset_path: str
    training_idx_path: str

GBUG_DATASET = Dataset(
    dataset_path="../artifacts/gbug-java.csv",
    training_idx_path="../artifacts/gbug-java_train_indexes.json",
)

DEFECT_DATASET = Dataset(
    dataset_path="../artifacts/defects4j.csv",
    training_idx_path="../artifacts/defects4j_train_indexes.json",
)

HUMAN_DATASET = Dataset(
    dataset_path="../artifacts/humaneval.csv",
    training_idx_path="../artifacts/humaneval_train_indexes.json",
)

In [ ]:

def get_testing_dataset(dataset: Dataset):
    """
    Get the testing dataset path and the training indexes path.
    """
    df = pd.read_csv(dataset.dataset_path)
    print(df.shape)
    with open( dataset.training_idx_path, "r") as f:
        training_indices = json.load(f)
    train_df = df.iloc[training_indices]
    test_df = df.drop(train_df.index)

    return train_df, test_df



In [ ]:
@dataclass
class SAE_ACTIVATIONS:
    feature_diff_path: str
    model_path: str
    layer: int
    top_k: int
    dataset: Dataset
    gbug_feature_safe_path: str
    gbug_feature_vuln_path: str
    defects_feature_safe_path: str
    defects_feature_vuln_path: str
    humaneval_feature_safe_path: str
    humaneval_feature_vuln_path: str

SAE_ACTIVATIONS_GBUG_LAYER_1 = SAE_ACTIVATIONS(
    feature_diff_path="../gpt2_gbug-java/layer1/feature_importance_diff.jsonl",
    gbug_feature_safe_path="../gpt2_gbug-java/layer1/feature_importance_safe.jsonl",
    gbug_feature_vuln_path="../gpt2_gbug-java/layer1/feature_importance_vuln.jsonl",
    defects_feature_safe_path="../gpt2_defects4j/layer1/feature_importance_safe.jsonl",
    defects_feature_vuln_path="../gpt2_defects4j/layer1/feature_importance_vuln.jsonl",
    humaneval_feature_safe_path="../gpt2_humaneval/layer1/feature_importance_safe.jsonl",
    humaneval_feature_vuln_path="../gpt2_humaneval/layer1/feature_importance_vuln.jsonl",
    model_path = "models/gbug_decision_tree_layer10_k_76.pt",
    layer = 10,
    top_k = 76,
    dataset = GBUG_DATASET,
)


In [ ]:

def read_jsonl_file(jsonl_path):
    with open(jsonl_path, "r") as f:
        for line in f:
            yield json.loads(line)

def get_feature_diff_path(dataset: Dataset, training_indexes):
    """
    Get the feature diff path for the given dataset.
    """
    import warnings
    warnings.filterwarnings("ignore")
    diff_data = list(read_jsonl_file(dataset.feature_diff_path))
    diff_df = pd.DataFrame(diff_data)

    columns = diff_df.columns.to_list()
    columns.remove("values")

    diff_df.drop(columns=columns, inplace=True)

    for i in trange(len(diff_df["values"][0])):
        diff_df[f"feature_{i}"] = diff_df["values"].apply(lambda x: x[i])

    diff_df.drop(columns=["values"], inplace=True)

    
    return diff_df



def get_most_important_features(train_df_diff, n=100):
    features = train_df_diff.sum(axis=0).sort_values(ascending=False).index[1 : n + 1]
    return features


In [ ]:
OUR_CONFIG = SAE_ACTIVATIONS_GBUG_LAYER_1

In [121]:
_, test_df = get_testing_dataset(OUR_CONFIG.dataset)

with open(OUR_CONFIG.dataset.training_idx_path, "r") as f:
    training_indices = json.load(f)

train_df_diff = get_feature_diff_path(
    SAE_ACTIVATIONS_GBUG_LAYER_1, training_indices
)

100%|██████████| 24576/24576 [00:34<00:00, 706.45it/s]


In [122]:

top_k = get_most_important_features(train_df_diff, n=OUR_CONFIG.top_k)

In [123]:
def get_vuln_safe_data(vuln_jsonl_path, safe_jsonl_path, train_indexes):
    vuln_data = list(read_jsonl_file(vuln_jsonl_path))
    safe_data = list(read_jsonl_file(safe_jsonl_path))
    vuln_df = pd.DataFrame(vuln_data)
    safe_df = pd.DataFrame(safe_data)
    vuln_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
    vuln_df["vuln"] = 1

    safe_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
    safe_df["vuln"] = 0

    for i in trange(len(vuln_df["values"][0])):
        vuln_df[f"feature_{i}"] = vuln_df["values"].apply(lambda x: x[i])
        safe_df[f"feature_{i}"] = safe_df["values"].apply(lambda x: x[i])

    safe_df_train = safe_df.loc[train_indexes]
    safe_df_test = safe_df.drop(train_indexes)

    vuln_df_train = vuln_df.loc[train_indexes]
    vuln_df_test = vuln_df.drop(train_indexes)

    df_train = pd.concat([safe_df_train, vuln_df_train])
    df_test = pd.concat([safe_df_test, vuln_df_test])

    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)
    df_train.drop(columns=["values"], inplace=True)
    df_test.drop(columns=["values"], inplace=True)

    return df_train, df_test


In [124]:
def evaluate_clf(clf, test_df):
    """
    Evaluate the model on the test dataset.
    """
    #bug_id	func_before	func_after
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i, row in tqdm(test_df.iterrows()):
        X = row[top_k.tolist()]
        y = row["vuln"]

        # Get the prediction
        pred = clf.predict([X])

        # Update the confusion matrix
        if pred == 1 and y == 1:
            tp += 1
        elif pred == 1 and y == 0:
            fp += 1
        elif pred == 0 and y == 1:
            fn += 1
        elif pred == 0 and y == 0:
            tn += 1


    # Calculate the accuracy
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    # Calculate the precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    # Calculate the recall
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    # Calculate the F1 score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn
    }

In [125]:
with open(SAE_ACTIVATIONS_GBUG_LAYER_1.model_path, "rb") as f:
    clf = pickle.load(f)

# Original (GBUG)
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.gbug_feature_vuln_path,
    OUR_CONFIG.gbug_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

100%|██████████| 24576/24576 [01:36<00:00, 255.43it/s]
60it [00:00, 4974.56it/s]


{'accuracy': 0.43333333333333335,
 'precision': 0.45,
 'recall': 0.6,
 'f1': 0.5142857142857143,
 'tp': 18,
 'fp': 22,
 'fn': 12,
 'tn': 8}

In [126]:
# Defects4J
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.defects_feature_vuln_path,
    OUR_CONFIG.defects_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

100%|██████████| 24576/24576 [01:47<00:00, 227.99it/s]
698it [00:00, 5400.30it/s]


{'accuracy': 0.5286532951289399,
 'precision': 0.5208333333333334,
 'recall': 0.7163323782234957,
 'f1': 0.6031363088057902,
 'tp': 250,
 'fp': 230,
 'fn': 99,
 'tn': 119}

In [127]:
# Defects4J
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.humaneval_feature_vuln_path,
    OUR_CONFIG.humaneval_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

100%|██████████| 24576/24576 [01:38<00:00, 249.19it/s]
88it [00:00, 4920.07it/s]


{'accuracy': 0.5,
 'precision': 0.5,
 'recall': 0.8409090909090909,
 'f1': 0.6271186440677967,
 'tp': 37,
 'fp': 37,
 'fn': 7,
 'tn': 7}